# Выбираем лучшую модель

Мы обучили дерево решений, случайный лес и линейную регрессию. Какая модель лучше?

## Задача

Найти модель, у которой на тестовой выборке RMSE не больше 7.5.

## Решение

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from joblib import dump

In [2]:
df = pd.read_csv('train_data.csv')

Разобьем данные на обучающую и валидационную выборки:

In [3]:
random_state = 12345

df_train, df_valid = train_test_split(df, test_size=0.25, random_state=random_state)

features_train = df_train.drop(['last_price'], axis=1)
target_train = df_train['last_price'] / 1_000_000

features_valid = df_valid.drop(['last_price'], axis=1)
target_valid = df_valid['last_price'] / 1_000_000

Обучим модели на обучающей выборке:

In [4]:
decision_tree = DecisionTreeRegressor(max_depth=10, random_state=random_state)
decision_tree.fit(features_train, target_train)

predicted_valid = decision_tree.predict(features_valid)

mse = mean_squared_error(target_valid, predicted_valid)
rmse = mse**0.5

print("RMSE на валидационной выборке:", rmse)

RMSE на валидационной выборке: 6.358309353080584


In [5]:
for estim in range(10, 51, 10):
    random_forest = RandomForestRegressor(n_estimators=estim, max_depth=10, random_state=random_state)
    random_forest.fit(features_train, target_train)
    predicted_valid = random_forest.predict(features_valid)
    
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse**0.5

    print("n_estimators =", estim, "RMSE на валидационной выборке:", rmse)

n_estimators = 10 RMSE на валидационной выборке: 8.047334972189107
n_estimators = 20 RMSE на валидационной выборке: 7.362362599066984
n_estimators = 30 RMSE на валидационной выборке: 7.080032280458895
n_estimators = 40 RMSE на валидационной выборке: 7.043174499132039
n_estimators = 50 RMSE на валидационной выборке: 7.226618080280562


In [6]:
linear_regression = LinearRegression()
linear_regression.fit(features_train, target_train)

predicted_valid = linear_regression.predict(features_valid)

mse = mean_squared_error(target_valid, predicted_valid)
rmse = mse**0.5

print("RMSE на валидационной выборке:", rmse)

RMSE на валидационной выборке: 7.726006697008546


Исходя из значений корня из среднеквадратичной ошибки (RMSE) выберем в качестве целевой модели **случайный лес в регрессии** с максимальной глубиной равной `10` и количеством деревьем равным `40`. Обучим ее на исходном датасете: чем больше данных, тем лучше результат. Валидационная выборка не нужна: вы уже определились с лучшей моделью.

In [7]:
features = df.drop(['last_price'], axis=1)
target = df['last_price'] / 1_000_000

In [8]:
best_model = RandomForestRegressor(n_estimators=40, max_depth=10, random_state=random_state)
best_model.fit(features, target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=40,
                      n_jobs=None, oob_score=False, random_state=12345,
                      verbose=0, warm_start=False)

In [9]:
predictions = best_model.predict(features)

mse = mean_squared_error(target, predictions)
rmse = mse**0.5

print("RMSE на исходном датасете:", rmse)

RMSE на исходном датасете: 3.4288390710078853


In [10]:
dump(best_model, 'best_model.joblib')

['best_model.joblib']

RMSE:

* Обучающая выборка: 3.4288390710078853
* Тестовая выборка: 7.005512842543898

Почему победила модель именно с такими гиперпараметрами? Вопрос, конечно же, интересный! Ответа на него нет. Иногда нужно просто перебирать разные варианты, смотреть на результаты, а потом взять — и выбрать лучшую модель.